# Updated Event Plotting

In [14]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [5]:
original_data_path = "/data/WatChMaL/data/IWCD_mPMT_Short_e-mu-gamma_E0to1000MeV_digihits.h5"
data_file = h5py.File(original_data_path, "r")


hdf5_hit_pmt    = data_file["hit_pmt"]
hdf5_hit_time   = data_file["hit_time"]
hdf5_hit_charge = data_file["hit_charge"]


hit_pmt    = np.memmap(original_data_path, mode="r", shape=hdf5_hit_pmt.shape,
                                    offset=hdf5_hit_pmt.id.get_offset(), dtype=hdf5_hit_pmt.dtype)

hit_time   = np.memmap(original_data_path, mode="r", shape=hdf5_hit_time.shape,
                                    offset=hdf5_hit_time.id.get_offset(), dtype=hdf5_hit_time.dtype)

hit_charge = np.memmap(original_data_path, mode="r", shape=hdf5_hit_charge.shape,
                                    offset=hdf5_hit_charge.id.get_offset(), dtype=hdf5_hit_charge.dtype)

angles     = np.array(data_file['angles'])
energies   = np.array(data_file['energies'])
positions  = np.array(data_file['positions'])
labels     = np.array(data_file['labels'])
root_files = np.array(data_file['root_files'])

In [8]:
barrel_map_array_idxs = [6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 15, 16, 17, 12, 13, 14, 18]
pmts_per_mpmt = 19

event_hits_index = np.append(data_file["event_hits_index"], hdf5_hit_pmt.shape[0]).astype(np.int64)

mpmt_positions_file = '/data/WatChMaL/data/IWCDshort_mPMT_image_positions.npz'
mpmt_positions   = np.load(mpmt_positions_file)['mpmt_image_positions']

data_size = np.max(mpmt_positions, axis=0) + 1
n_channels = pmts_per_mpmt
data_size = np.insert(data_size, 0, n_channels)

In [19]:
print(mpmt_positions.shape)

(536, 2)


In [ ]:
def PMT_to_flat_cylinder_mapping( tubes, tube_xyz ):
    """
    Build dictionary of PMT number, to (x,y) on a flat cylinder
    
    N.B. Tube numbers in full geometry file go from 1:NPMTs, but it seems like
    the event data number from 0:NPMTs-1, so subtracting 1 from tube number here?
    """
    mapping = {}
    for idx, tube in enumerate(tubes):
        x = tube_xyz[idx,0]
        y = tube_xyz[idx,1]
        z = tube_xyz[idx,2]
        if ( y > 500. ):
            # in top circle of cylinder
            xflat = x
            yflat = 897.6 + z
            mapping[ int( tube-1 ) ] = [ xflat, yflat ]
            
        elif ( y < -500.):
            # in bottom circle of cylinder+
            xflat = x
            yflat = -897.6 + z
            mapping[ int( tube-1 ) ] = [ xflat, yflat ]
            
        else:
            # in barrel part of cylinder
            theta = math.atan2( z, x )
            xflat = R * theta
            yflat = y
            mapping[ int( tube-1 ) ] = [ xflat, yflat ]
    return mapping

In [ ]:
PMTFlatMapping = PMT_to_flat_cylinder_mapping( tubes, tube_xyz )

In [9]:
def get_event_data_from_index(item):
    start = event_hits_index[item]
    stop  = event_hits_index[item + 1]

    hit_pmts    = hit_pmt[start:stop].astype(np.int16)
    hit_charges = hit_charge[start:stop]
    hit_times   = hit_time[start:stop]

    hit_mpmts = hit_pmts // pmts_per_mpmt
    hit_pmt_in_modules = hit_pmts % pmts_per_mpmt

    hit_rows = mpmt_positions[hit_mpmts, 0]
    hit_cols = mpmt_positions[hit_mpmts, 1]

    sample_data = np.zeros(data_size)
    sample_data[hit_pmt_in_modules, hit_rows, hit_cols] = hit_charges

    # fix barrel array indexing to match endcaps in xyz ordering
    sample_data[:, 12:28, :] = sample_data[barrel_map_array_idxs, 12:28, :]
    moved_data = np.moveaxis(sample_data, 0, -1)
    return moved_data

In [12]:
def EventDisplay( tubes, quantities, title="Charge", cutrange=[-1,-1] ):
    """
    tubes == np.array of PMTs that were hit
    quantities == np.array of PMT quantities (either charge or time)
    title == title to add to display
    cutrange == minimum and maximum values on plot (or set both same for default)
    """
    
    fig = plt.figure(figsize=[12,12]) 
    preimage = np.zeros( [2506, 2317] )
    #maxquantity = quantities.max()
    #preimage *= maxquantity*1.2
    imgmin = quantities.min()
    imgmax = quantities.max()
    for idx, tube in enumerate( tubes ):
        if cutrange[0] != cutrange[1]:
            if quantities[idx] < cutrange[0] or quantities[idx] > cutrange[1]:
                continue
        for dx in range(-3,4):
            for dy in range(-3,4):
                if abs(dx)==3 and abs(dy)==3:
                    continue
                    
                #print( "idx=", idx, " len(quantities)=",len(quantities), " tube=", tube, " len(PMTFlatMap)=", len(PMTFlatMapPositive))
                preimage[ PMTFlatMapPositive[tube][1]+dx, PMTFlatMapPositive[tube][0]+dy ] = quantities[idx]

    if cutrange[0] != cutrange[1]:
        imgmin = cutrange[0]
        imgmax = cutrange[1]
    
    plt.imshow( preimage, extent = [-1162.7,1162.7,-1267.7,1267.7], vmin=imgmin, vmax=imgmax )
    fig.suptitle(title, fontsize=20)
    plt.xlabel('Distance CCW on perimeter from x-axis (cm)', fontsize=18)
    plt.ylabel('Y (cm)', fontsize=16)
    
    #plt.set_cmap('YlGnBu')
    #plt.set_cmap('cubehelix_r')
    #plt.set_cmap('gnuplot2_r')
    
    plt.set_cmap('gist_heat_r')
    plt.colorbar()



In [10]:
test_event_data = get_event_data_from_index(0)

In [15]:
EventDisplay(mpmt_positions, test_event_data)

NameError: name 'PMTFlatMapPositive' is not defined

<Figure size 864x864 with 0 Axes>

In [17]:
datafile = np.load('/data/WatChMaL/data/IWCDmPMT_4pi_full_tank_mu-_E0to1000MeV_unif-pos-R371-y521cm_4pi-dir_3000evts_25.npz', allow_pickle=True)
geofile = np.load('full_geo_dump.npz',allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: '/data/WatChMaL/data/IWCDmPMT_4pi_full_tank_mu-_E0to1000MeV_unif-pos-R371-y521cm_4pi-dir_3000evts_25.npz'

In [ ]:
tube_xyz = geofile[ 'position' ]
